In [14]:
import pandas as pd
import requests
import datetime
import math

# Global token map
token_df = pd.DataFrame()

def initializeSymbolTokenMap():
    global token_df
    url = "https://margincalculator.angelbroking.com/OpenAPI_File/files/OpenAPIScripMaster.json"
    d = requests.get(url).json()
    token_df = pd.DataFrame.from_dict(d)
    token_df["expiry"] = pd.to_datetime(token_df["expiry"], errors="coerce")
    token_df["strike"] = pd.to_numeric(token_df["strike"], errors="coerce")
    print("Symbol Token Map Initialized")

def getTokenInfo(exch_seg, instrumenttype, symbol, strike_price=None, pe_ce=None):
    global token_df
    df = token_df

    if exch_seg == "NSE" and instrumenttype == "EQ":
        return df[
            (df["exch_seg"] == "NSE") &
            (df["symbol"] == symbol)
        ]

    elif exch_seg == "NFO" and instrumenttype == "FUTIDX":
        return df[
            (df["exch_seg"] == "NFO") &
            (df["instrumenttype"] == instrumenttype) &
            (df["name"] == symbol)
        ]

    elif exch_seg == "NFO" and instrumenttype in ["OPTSTK", "OPTIDX"]:
        filtered = df[
            (df["exch_seg"] == "NFO") &
            (df["instrumenttype"] == instrumenttype) &
            (df["name"] == symbol) &
            (df["option_type"] == pe_ce)
        ]
        if strike_price is not None:
            filtered = filtered[filtered["strike"] == float(strike_price)]
        return filtered

    return pd.DataFrame()

# Run logic
initializeSymbolTokenMap()

# Get BANKNIFTY token from NSE segment
nifty_token_info = getTokenInfo("NSE", "EQ", "NIFTY")
  # 🔹 Mock LTP — replace this with live API response later
# ltp = 49025  # Simulated value
# rtm = round(ltp / 100) * 100  # Round to nearest 100

# 🔹 Get CE/PE at that strike
# ce_info = getTokenInfo("NFO", "OPTIDX", "BANKNIFTY", rtm, "CE")
# pe_info = getTokenInfo("NFO", "OPTIDX", "BANKNIFTY", rtm, "PE")

if not nifty_token_info.empty:
    nifty_token = nifty_token_info.iloc[0]["token"]
    # ce_info_token = ce_info.iloc[0]["token"]
    # pe_info_token = pe_info.iloc[0]["token"]
    print("BANKNIFTY Token:", nifty_token)
    # print("BANKNIFTY CE Token:", ce_info_token)
    # print("BANKNIFTY PE Token:", pe_info_token)
else:
    print("BANKNIFTY token not found.")


Symbol Token Map Initialized
BANKNIFTY Token: 26000


C:\Users\Admin\AppData\Local\Temp\ipykernel_12560\3961665370.py:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  token_df["expiry"] = pd.to_datetime(token_df["expiry"], errors="coerce")
